# Transfer learning:


Transfer learning is a popular technique in deep learning that allows the use of pre-trained neural network models to solve new tasks. By leveraging the knowledge learned from one task, a pre-trained model can be fine-tuned on a different, but related task, thus saving time and computational resources. PyTorch is a popular deep learning framework that provides powerful tools and libraries for building and training neural networks, including pre-trained models that can be used for transfer learning. With PyTorch, transfer learning can be easily implemented, enabling developers to quickly build high-performing models for a wide range of applications.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [25]:
import os
# from tqdm import tqdm
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F  
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import copy
from torchsummary import summary
#Confusion matrix:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import itertools


In [26]:
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler

In [27]:


import sys
import os

py_file_location = "/content/drive/MyDrive/"
sys.path.append(os.path.abspath(py_file_location))

In [28]:

import utils

In [ ]:
# define device
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [29]:
DATA_DIR = '/content/drive/MyDrive/fma/data/fma_small'

tracks = utils.load('/content/drive/MyDrive/fma/data/fma_metadata/tracks.csv')
features = utils.load('/content/drive/MyDrive/fma/data/fma_metadata/features.csv')#annotation files
echonest = utils.load('/content/drive/MyDrive/fma/data/fma_metadata/echonest.csv')

subset = tracks.index[tracks['set', 'subset'] <= 'small']

assert subset.isin(tracks.index).all()
assert subset.isin(features.index).all()

features_all = features.join(echonest, how='inner').sort_index(axis=1)
print('Not enough Echonest features: {}'.format(features_all.shape))

tracks = tracks.loc[subset]
features_all = features.loc[subset]

tracks.shape, features_all.shape

train = tracks.index[tracks['set', 'split'] == 'training'] #bunch of indexes (not ids) for training val and test
val = tracks.index[tracks['set', 'split'] == 'validation']
test = tracks.index[tracks['set', 'split'] == 'test']

Not enough Echonest features: (13129, 767)


In [ ]:
len(train)

In [ ]:
tracks_index = tracks.index
tracks_index

In [32]:
#From data to one hot labels
labels_onehot = LabelBinarizer().fit_transform(tracks['track', 'genre_top'])
labels_onehot_Ten = torch.tensor(labels_onehot)
labels_onehot = pd.DataFrame(labels_onehot, index=tracks.index)




#from onehot labels to encoded targets.
targets = torch.argmax(labels_onehot_Ten, dim=1)





In [ ]:
#creating an annotation dataframe.


df = pd.read_csv('/content/drive/MyDrive/fma/data/fma_small/checksums', sep='  |/', header = None,
                 names = ['id', 'fold', 'songs'], converters={'fold': str})
df.index = tracks_index
df.loc[5][1]

In [34]:
#adding the ch_three attribute/option to create 3 channel spectrogram.

#for manual spectogram we used one channel, but for the prepared one from pytorch we used 3 channel spectogram

#three channels are simply just a 3 replicas of 1 channel spectrogram.





from torch.utils.data import Dataset, DataLoader


#custome dataset class
class FMA(Dataset):
    def __init__(self, data_dir, track_ids, annotation,
                 target_sample_rate, transformation, num_samples, device = False , twoD = False, paper_cut = False):
        self.annotation = annotation
        self.data_dir = data_dir
        self.track_ids = track_ids
        self.filenames = os.listdir(data_dir)
        self.target_sample_rate = target_sample_rate
        self.device = device
        self.transformation = transformation
        if self.device == True :
          self.transformation = transformation.to(self.device)
        self.twoD = twoD
        self.num_samples = num_samples
        self.paper_cut = paper_cut


        
        

    def __getitem__(self, index):
        tid = self.track_ids[index]
        filepath = self._get_audio_sample_path(tid)
        label = torch.from_numpy(labels_onehot.loc[tid].values).float()
        
        try:
            waveform, sr = torchaudio.load(filepath)
        except:
            print(filepath)
        #be careful all of the sample rates aren't the same(resample)
        #waveform --> (2, 10000) #(number of channels, number of samples)
        if self.device == True :
          waveform = waveform.to(self.device)
        waveform = self._resample_if_necessary(waveform, sr)
        waveform = self._mix_down_if_necessary(waveform)
        #we have to adjust the length of the audio waveforms before the transformation
        waveform = self._cut_if_necessary(waveform)
        waveform = self._right_pad_if_necessary(waveform)
        if self.twoD == True:
            waveform = self.transformation(waveform)
        else:
            pass
        
        
        if self.paper_cut == True:
            waveform = waveform[:, :128, :513]
        else:
            pass
        
        

        return waveform, label
    
    
    def _get_audio_sample_path(self, dex):
        fold = self.annotation.loc[dex][1]
        path = os.path.join(self.data_dir, fold, self.annotation.loc[dex][2])
        return path
        

            
    
    
    
    def _cut_if_necessary(self, waveform):
        #this method happens before the transformation
        if waveform.shape[1] > self.num_samples:
            waveform = waveform[:, :self.num_samples]
            return waveform
        
        
    def _right_pad_if_necessary(self, waveform):
        if waveform.shape[1] < self.num_samples:
            num_missing_samples = self.num_samples - waveform.shape[1]
            last_dim_padding = (0,num_missing_samples) # (1, 2) -> (left, right)   
            #(1, 2, 0, 1) -> (left, right, padnumleft, padnumright)
            # what happens is : [1, 1, 1] --> [0, 1, 1, 1, 0, 0]
            waveform = torch.nn.functional.pad(waveform, last_dim_padding)
            waveform = waveform.T
        return waveform
    
    
        
    def _resample_if_necessary(self, waveform , sr):
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate)
            waveform = resampler(waveform)
        return waveform
    
    
    #from (2, 10000) to (1, 0000) taking the average between two waveforms
    def _mix_down_if_necessary(self, waveform):
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform , dim = 0, keepdim = True)
        return waveform
     
    

        
        return waveform, label 
    
    def __len__(self):#just gives us the number of samples in our datasets.
        return len(self.track_ids) 

        

        

In [ ]:
#trying the class:

if __name__ == "__main__":
    

    SAMPLE_RATE=44100

    #maxlength
    NUM_SAMPLES = 44100
    #working on GPU
 #   if torch.cuda.is_available():
 #       Device = "cuda"
 #   else:
 #       Device = "cpu"
        
 #   print(f"we are using {Device}.")  
    
    #50% hop_length is the best for accuracy
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(sample_rate = SAMPLE_RATE, n_fft = 1024, hop_length = 256,
                                                        n_mels = 64) 
    
    n_fft = 1024    # FFT window size
    hop_length = 256    # number of samples between successive frames
    win_length = n_fft
    
    spectrogram = torchaudio.transforms.Spectrogram(n_fft=1024, hop_length = 256, win_length = win_length )
    
    
    
    FL = FMA(DATA_DIR, train, df, SAMPLE_RATE, mel_spectrogram, NUM_SAMPLES, twoD =True)
    print(f"there are {len(FL)} samples in the dataset" )
    waveform, label = FL[0] #track number 2
    a = 1
    
    


In [ ]:
#Here are corruted songs!
#it's your choice how to deal with the :)

Dex = tracks_index

def _get_audio_sample_path(data_dir, dex): 
        fold = df.loc[dex][1]
        path = os.path.join(data_dir, fold, df.loc[dex][2])
        return path
    
for i in Dex:
    p = _get_audio_sample_path(DATA_DIR, i)
    try:
            w, sr = torchaudio.load(p)
    except:
            print(p)
    

# VGG16 without weights

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=False)
model.classifier[6] = nn.Sequential(nn.Linear(in_features=4096, out_features=8, bias=True), nn.Dropout(0.5))
model.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
model

In [ ]:
BATCH = 32


FL = FMA(DATA_DIR, train, df, SAMPLE_RATE, mel_spectrogram, NUM_SAMPLES, twoD =True, paper_cut = True)
val_dataset = FMA(DATA_DIR, val,df, SAMPLE_RATE, mel_spectrogram, NUM_SAMPLES, twoD =True, paper_cut = True)



val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH, shuffle=True)
dataloader = torch.utils.data.DataLoader(FL, batch_size=BATCH, shuffle=True)



#model.to(device)


loss_fn = nn.CrossEntropyLoss()

# Adam optimizer
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001)

# Define the scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=5)



num_epochs = 10
i = 0
running_loss = 0.0



train_acV_no = []
val_acV_no = [] 


# train the model
for epoch in range(num_epochs):
    # evaluate the model on the training dataset
    model.train()
    train_correct = 0
    train_total = 0
    for waveform, label in dataloader:
        #label = label.to(device)
        train_label = torch.argmax(label, dim=1)

        # clear the gradients
        optimizer.zero_grad()

        # forward pass
        waveform = waveform.squeeze(0)


        
        #waveform = waveform.to(device)
        output = model(waveform)
        

        

        loss = loss_fn(output, label)

        # backward pass
        loss.backward()
        optimizer.step()  
        
        # Update the learning rate
        scheduler.step(loss)
            
        _, train_predicted = torch.max(output.data, 1)
        train_total += train_label.size(0)
        train_correct += (train_predicted == train_label).sum().item()
        # print statistics
        i += 1
        running_loss += loss.item()
        
        
    train_a = train_correct / train_total        
    train_acV_no.append(train_a)       
    print('[%d, %5d subsamples] Training loss: %.3f' % (epoch + 1, i*BATCH, running_loss / len(dataloader)))
    running_loss = 0            
    # evaluate the model on the validation dataset
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        model.eval()
        for val_waveform, val_label in val_dataloader:
            #val_label = val_label.to(device)
            val_label = torch.argmax(val_label, dim=1)
            val_waveform = val_waveform.squeeze(0)
            
            #val_waveform = val_waveform.to(device)
            val_output = model(val_waveform)
            val_loss += loss_fn(val_output, val_label).item()
            _, val_predicted = torch.max(val_output.data, 1)
            val_total += val_label.size(0)
            val_correct += (val_predicted == val_label).sum().item()
            
            
    val_a = val_correct/ val_total
    val_acV_no.append(val_a)
    print('Validation Loss: {:.4f} | Validation Accuracy: {:.4f} | Training Accuracy: {:.4f}'.format(val_loss / len(val_dataloader), val_correct / val_total, train_correct / train_total))
print('Finished Training')

In [ ]:
import json


with open("/content/drive/MyDrive/Result/train_acV_no.json", 'w') as f:

    json.dump(train_acV_no, f, indent=2) 

with open("/content/drive/MyDrive/Result/val_acV_no.json", 'w') as f:

    json.dump(val_acV_no, f, indent=2) 

# VGG16 with pytorch pretrained weights:

when using a pretrained model, it's important that your custom data going into the model is prepared in the same way as the original training data that went into the model.

In [ ]:
import torchvision

weights = torchvision.models.VGG16_Weights.DEFAULT
model = torchvision.models.vgg16(weights=weights)

In [ ]:
model

In [ ]:
#We have to change the first layer, and last layer of the networkbc our data has 1 channel and 8 outputs.
#but we are going to freeze other layers.
for i in range(1, 31):
    for param in model.features[i].parameters():
        param.requires_grad = False

for j in range(0, 6):
  for param in model.classifier[j].parameters():
    param.requires_grad = False

In [ ]:
#changing the first cnn and last linear layer:

model.classifier[6] = nn.Sequential(nn.Linear(in_features=4096, out_features=8, bias=True), nn.Dropout(0.5))
model.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
model

In [ ]:
BATCH = 32


FL = FMA(DATA_DIR, train, df, SAMPLE_RATE, mel_spectrogram, NUM_SAMPLES, twoD =True, paper_cut = True)
val_dataset = FMA(DATA_DIR, val,df, SAMPLE_RATE, mel_spectrogram, NUM_SAMPLES, twoD =True, paper_cut = True)



val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH, shuffle=True)
dataloader = torch.utils.data.DataLoader(FL, batch_size=BATCH, shuffle=True)



#model.to(device)


loss_fn = nn.CrossEntropyLoss()

# Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Define the scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=5)



num_epochs = 10
i = 0
running_loss = 0.0



train_acV_pyt = []
val_acV_pyt = [] 


# train the model
for epoch in range(num_epochs):
    # evaluate the model on the training dataset
    model.train()
    train_correct = 0
    train_total = 0
    for waveform, label in dataloader:
        #label = label.to(device)
        train_label = torch.argmax(label, dim=1)

        # clear the gradients
        optimizer.zero_grad()

        # forward pass
        waveform = waveform.squeeze(0)


        
        #waveform = waveform.to(device)
        output = model(waveform)
        

        

        loss = loss_fn(output, label)

        # backward pass
        loss.backward()
        optimizer.step()  
        
        # Update the learning rate
        scheduler.step(loss)
            
        _, train_predicted = torch.max(output.data, 1)
        train_total += train_label.size(0)
        train_correct += (train_predicted == train_label).sum().item()
        # print statistics
        i += 1
        running_loss += loss.item()
        
        
    train_a = train_correct / train_total        
    train_acV_pyt.append(train_a)       
    print('[%d, %5d subsamples] Training loss: %.3f' % (epoch + 1, i*BATCH, running_loss / len(dataloader)))
    running_loss = 0            
    # evaluate the model on the validation dataset
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        model.eval()
        for val_waveform, val_label in val_dataloader:
            #val_label = val_label.to(device)
            val_label = torch.argmax(val_label, dim=1)
            val_waveform = val_waveform.squeeze(0)
            
            #val_waveform = val_waveform.to(device)
            val_output = model(val_waveform)
            val_loss += loss_fn(val_output, val_label).item()
            _, val_predicted = torch.max(val_output.data, 1)
            val_total += val_label.size(0)
            val_correct += (val_predicted == val_label).sum().item()
            
            
    val_a = val_correct/ val_total
    val_acV_pyt.append(val_a)
    print('Validation Loss: {:.4f} | Validation Accuracy: {:.4f} | Training Accuracy: {:.4f}'.format(val_loss / len(val_dataloader), val_correct / val_total, train_correct / train_total))
print('Finished Training')

In [ ]:
import json

with open("/content/drive/MyDrive/Result/train_acV_pyt.json", 'w') as f:

    json.dump(train_acV_pyt, f, indent=2) 

with open("/content/drive/MyDrive/Result/val_acV_pyt.json", 'w') as f:

    json.dump(val_acV_pyt, f, indent=2) 

# VGG16 with GTZAN pretrained weights:

### GTZAN dataset:
The GTZAN dataset is a widely used benchmark dataset in the field of music information retrieval. It was created by George Tzanetakis and Perry Cook in 2002, and is named after Tzanetakis' initials. The dataset consists of 1,000 audio tracks of 30 seconds each, equally divided into 10 different music genres: blues, classical, country, disco, hip-hop, jazz, metal, pop, reggae, and rock. The audio tracks were collected from various online sources and were pre-processed to have a uniform format and sampling rate. The GTZAN dataset has been extensively used for various music classification tasks, such as genre classification, mood detection, and instrument recognition. Its popularity stems from its wide variety of genres and large sample size, which makes it a valuable resource for developing and evaluating music analysis algorithms.

In [ ]:
import torch
import torchvision.models as models

model = models.vgg16(pretrained=False)
model.classifier[6] = nn.Linear(in_features=4096, out_features=10, bias=True)
model.load_state_dict(torch.load('/content/drive/MyDrive/Result/model_weights.pth'))


In [37]:
#We have to change the first layer, and last layer of the networkbc our data has 1 channel and 8 outputs.
#but we are going to freeze other layers.
for i in range(1, 31):
    for param in model.features[i].parameters():
        param.requires_grad = False

for j in range(0, 6):
  for param in model.classifier[j].parameters():
    param.requires_grad = False

In [ ]:

#changing the first cnn and last linear layer:

model.classifier[6] = nn.Sequential(nn.Linear(in_features=4096, out_features=8, bias=True), nn.Dropout(0.5))
model.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
BATCH = 32


FL = FMA(DATA_DIR, train, df, SAMPLE_RATE, mel_spectrogram, NUM_SAMPLES, twoD =True, paper_cut = True)
val_dataset = FMA(DATA_DIR, val,df, SAMPLE_RATE, mel_spectrogram, NUM_SAMPLES, twoD =True, paper_cut = True)



val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH, shuffle=True)
dataloader = torch.utils.data.DataLoader(FL, batch_size=BATCH, shuffle=True)



model.to(device)


loss_fn = nn.CrossEntropyLoss()

# Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Define the scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=5)



num_epochs = 10
i = 0
running_loss = 0.0



train_acV_G = []
val_acV_G = [] 


# train the model
for epoch in range(num_epochs):
    # evaluate the model on the training dataset
    model.train()
    train_correct = 0
    train_total = 0
    for waveform, label in dataloader:
        label = label.to(device)
        waveform = waveform.to(device)
        train_label = torch.argmax(label, dim=1)

        # clear the gradients
        optimizer.zero_grad()

        # forward pass
        waveform = waveform.squeeze(0)


        
        #waveform = waveform.to(device)
        output = model(waveform)
        

        

        loss = loss_fn(output, label)

        # backward pass
        loss.backward()
        optimizer.step()  
        
        # Update the learning rate
        scheduler.step(loss)
            
        _, train_predicted = torch.max(output.data, 1)
        train_total += train_label.size(0)
        train_correct += (train_predicted == train_label).sum().item()
        # print statistics
        i += 1
        running_loss += loss.item()
        
        
    train_a = train_correct / train_total        
    train_acV_G.append(train_a)       
    print('[%d, %5d subsamples] Training loss: %.3f' % (epoch + 1, i*BATCH, running_loss / len(dataloader)))
    running_loss = 0            
    # evaluate the model on the validation dataset
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        model.eval()
        for val_waveform, val_label in val_dataloader:
            val_label = val_label.to(device)
            val_label = torch.argmax(val_label, dim=1)
            val_waveform = val_waveform.squeeze(0)
            
            val_waveform = val_waveform.to(device)
            val_output = model(val_waveform)
            val_loss += loss_fn(val_output, val_label).item()
            _, val_predicted = torch.max(val_output.data, 1)
            val_total += val_label.size(0)
            val_correct += (val_predicted == val_label).sum().item()
            
            
    val_a = val_correct/ val_total
    val_acV_G.append(val_a)
    print('Validation Loss: {:.4f} | Validation Accuracy: {:.4f} | Training Accuracy: {:.4f}'.format(val_loss / len(val_dataloader), val_correct / val_total, train_correct / train_total))
print('Finished Training')

In [ ]:
import json

with open("/content/drive/MyDrive/Result/train_acV_G.json", 'w') as f:

    json.dump(train_acV_G, f, indent=2) 

with open("/content/drive/MyDrive/Result/val_acV_G.json", 'w') as f:

    json.dump(val_acV_G, f, indent=2) 